<h1 style='color:red; text-align:center;'>Vendor Network Optimization for City-wide Product Delivery</h1>

<h2>Introduction</h2>
<p>
This project aims to enhance the 
    <span style='font-weight:bold;'>
        efficiency of product delivery 
    </span>
    across a city by optimizing the 
    <span style='font-weight: bold;'>
    vendor network.
    </span>
        
    
</br>
    Our primary focus is to 
    <span style='color:red;'>
        maximize
    </span> 
    <span style='font-weight: bold;'>
        city-wide coverage
    </span>
    by strategically selecting vendors who offer the 
    <span style='font-weight:bold;'>
        best combination of 
        <span style='color:red;'>
            high performance
        </span>and 
        <span style='color:red;'>
            customer satisfaction
        </span>
    </span>.
    
</p>

<ul>
    <li>
        <span style='font-weight: bold;'>
            Vendor Network Optimization
        </span>
        <p>
            The objective is to build a network of vendors that maximizes customer coverage. Coverage is defined as the proportion of customer sessions within a 2 km radius of any vendor. 
            The key goals of this part include:
            <ul>
                <li>Maximizing the number of customer sessions covered by vendors.</li>
                <li>Prioritizing vendors with higher customer satisfaction ratings, based on performance scores.</li>
                <li>Ensuring that certain mandatory vendors are included while excluding blacklisted vendors.</li>
            </ul>
        </p>
    </li>
</ul>
</br>
<p>
    <span style='font-weight: bold; color:red;'>
        Note:
    </span>
    Based on company rules, I can't use actual data, so I create sample datas and run code for them.
</p>

In [51]:
!pip install pandas numpy folium geopy scipy pulp

In [53]:
# read and analize data
import numpy as np
import pandas as pd

import folium # map drawing

# optimizing
from geopy.distance import geodesic
from scipy.sparse import csr_matrix
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

In [26]:
# read data
vendors = pd.read_csv('DataSets/sample_vendors.csv')
sessions = pd.read_csv('DataSets/sample_sessions.csv')
print('vendor columns:', list(vendors.columns))
print('session columns:', list(sessions.columns))

vendor columns: ['ID', 'latitude', 'longitude', 'Score', 'Status']
session columns: ['latitude', 'longitude', 'Count']


<p style='font-weight: bold;'>We use forium for drawing vendors on the map.</p>

In [46]:
vendor_map = folium.Map(location=[vendors.latitude.mean(), vendors.longitude.mean()], zoom_start=10) # set camera to center of vendors

# set one mark to each vendor
for i in range(len(vendors)):
    temp = vendors.iloc[i]
    folium.Marker(
        location=[temp.latitude, temp.longitude], # set marker
        popup=f"Vendor ID: {temp.ID}, Performance: {temp.Score}", # infor of marker
        icon=folium.Icon(color="blue" if temp.Status == 1 else "red" if temp.Status == -1 else "green") # color of marker
    ).add_to(vendor_map)


vendor_map

<p>
    For creating network between vendors and coustemers, we need to find distance matrix between vendors and sessions.
</p>

In [65]:
def calculate_distance_matrix(vendors, sessions):
    """
        calculate distance matrix for vendors and sessions
    
    """
    vendor_cor = vendors[['latitude', 'longitude']].values
    session_cor = sessions[['latitude', 'longitude']].values
    distance_matrix = np.zeros((vendors.shape[0], sessions.shape[0]))

    for i, v_coord in enumerate(vendor_cor):
        for j, s_coord in enumerate(session_cor):
            distance_matrix[i, j] = geodesic(v_coord, s_coord).km # find distance of two cord with goe library

    return distance_matrix


In [67]:
dist_mat = calculate_distance_matrix(vendors, sessions) # create distance matrix for data

<p>
    Now we can eliminate the distances that not valid for our problem.
</p>

In [138]:
coverage_matrix = (dist_mat <= 2).astype(int) # cov matrix(without optimizinfg)
session_count_matrix = coverage_matrix * sessions['Count'].values
vendor_coverage_mat = session_count_matrix.sum(axis=1)

<p>
    now we have vendor coverage, so we can optmiize it by score and vendor selection limit.
</br>
    for this we can use linear programming methods.
</p>

In [142]:
def optimize_vendors(vendor_df, vendor_coverage, num_vendors_to_select=150):
    """
        optmiize vendor by number limit ans score
    """

    prob = LpProblem("vendor_network_optimizing", LpMaximize)
    x_vars = LpVariable.dicts("vendor", vendor_df.index, cat='Binary') # decision variable.
    
    prob += lpSum([x_vars[i] * (vendor_coverage[i] + 0.1 * vendor_df.loc[i, 'Score'])
                   for i in vendor_df.index]) # score and coverage maximizing.

    prob += lpSum([x_vars[i] for i in vendor_df.index]) == num_vendors_to_select # select 150

    mandatory_vendors = vendor_df[vendor_df['Status'] == 1].index
    for v in mandatory_vendors:
        prob += x_vars[v] == 1 # select status==1

    restricted_vendors = vendor_df[vendor_df['Status'] == -1].index
    for v in restricted_vendors:
        prob += x_vars[v] == 0  # restrict status==-1

    prob.solve()
    
    return [i for i in vendor_df.index if x_vars[i].varValue == 1]

In [144]:
selected_vendors = optimize_vendors(vendors, vendor_coverage_mat)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fk/52xy4m450vz2_jxd2k7_k5n00000gn/T/ca3721eda4ac47459dc51b5f50e47053-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fk/52xy4m450vz2_jxd2k7_k5n00000gn/T/ca3721eda4ac47459dc51b5f50e47053-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 111 COLUMNS
At line 2217 RHS
At line 2324 BOUNDS
At line 2825 ENDATA
Problem MODEL has 106 rows, 500 columns and 605 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 56505.7 - 0.00 seconds
Cgl0002I 50 variables fixed
Cgl0004I processed model has 1 rows, 395 columns (395 integer (395 of which binary)) and 395 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -56505.7
Cbc0038I Before mini branch and bo